In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import healpy as hp
from statistics import mean, stdev
import os

output_dir = "Randomized_data"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

nside = 8   # this is for masking 1 and masking 2
nf = 100  # number of samples


for k in range(3):
    f = '../data_prep/mask1/masked_sample_' + str(k+1) + '.dat'
    # f = '../data_prep/mask2/masked_sample_' + str(k+1) + '.dat'
    
    df1 = pd.read_csv(f,sep="\t",header = None)
    df1.columns = ['r','th','ph']

    r = df1['r'].to_numpy()    # in Mpc
    th = df1['th'].to_numpy()
    ph = df1['ph'].to_numpy()
   
#####################################################################################
    f1 = '../data_prep/mask1/non_zero_pix_id_' + str(k+1) + '.dat'
    # f1 = '../data_prep/mask2/non_zero_pix_id_' + str(k+1) + '.dat'
   

    df4 = pd.read_csv(f1,sep="\t",header = None)
    df4.columns = ['id_list']
    id_pix = df4['id_list'].to_numpy()
#####################Randomization###################################################
    max_cos= 1.0
    min_cos = -1.0

    for f in range(nf):
        
        random.seed((f+1))
        num_points = len(r)
    
        valid_theta = []
        valid_phi = []
        
        while(len(valid_theta) < num_points):
            cos_theta = random.uniform(min_cos, max_cos)  
            phi = random.uniform(0, 2*np.pi) 
            
            theta = np.arccos(cos_theta)
            pix = hp.ang2pix(nside, theta, phi)
            
            if pix in id_pix:
                valid_theta.append(theta)
                valid_phi.append(phi)
     
        v_theta = np.array(valid_theta)
        v_phi = np.array(valid_phi)

        dict1 = {'r': r, 'th': v_theta, 'ph': v_phi } 
        fd = pd.DataFrame(dict1)

        out = 'Randomized_data/s' + str(k+1) + '_rand' + str(f+1) + '.dat'
        fd.to_csv(out, sep="\t", header=None, index=False)

In [2]:
# Compute Rényi entropies for q = 1 to 5

def compute_renyi_entropy(input_file, output_file,rmin, rmax, Nside=nside, nbin=30):
    Npix = hp.nside2npix(Nside)

    # Load data
    df = pd.read_csv(input_file, sep="\t", header=None)
    df.columns = ['r', 'th', 'ph']
    r1 = df['r'].to_numpy()
    th1 = df['th'].to_numpy()
    ph1 = df['ph'].to_numpy()

    # Set up radial bins    compute_renyi_entropy(f_in, f_out1)

    #rmin, rmax = np.min(r1), np.max(r1)
    dr = (rmax - rmin) / nbin

    # Initialize counts and effective numbers
    m = np.zeros((nbin, Npix), dtype=float)
    Neff = np.zeros(nbin, dtype=int)

    # Fill pixel counts per radial bin
    for rr, tt, pp in zip(r1, th1, ph1for_github_g_20for_github_g_20):
        for j in range(nbin):
            if rmin <= rr <= (rmin + (j+1) * dr):
                px = hp.ang2pix(Nside, tt, pp)
                m[j][px] += 1
                Neff[j] += 1

    # Normalize to get probabilities
    p = np.zeros((nbin, Npix), dtype=float)
    for i in range(nbin):
        for px in range(Npix):
            if Neff[i] > 0:
                p[i][px] = m[i][px] / Neff[i]

    H = np.zeros((nbin, 5), dtype=float)
    a = np.zeros((nbin, 5), dtype=float)
    
    for k in range(5):
        h = np.zeros(nbin, dtype=float)
        q = k + 1
        if q == 1:
            for i in range(nbin):
                for px in range(Npix):
                    if p[i][px] > 0:
                        H[i][k] -= p[i][px] * np.log10(p[i][px])
        else:
            for i in range(nbin):
                for px in range(Npix):
                    h[i] += p[i][px] ** q
                if h[i] > 0:
                    H[i][k] = np.log10(h[i]) / (1 - q)

        for i in range(nbin):
            a[i][k] = H[i][k]

    # Construct output: radius and entropy values
    R = np.zeros(nbin)
    for b in range(nbin):
        R[b] = rmin + (b+1)*dr
        
    df1=pd.DataFrame(data=a)
    df2=pd.DataFrame(data=R)
    df3= pd.concat([df2,df1], axis=1, join='inner')
    df3.to_csv(output_file, sep='\t', header=False, index=False)
    
#########################################################################################


R_max = np.zeros(3)
R_min = np.zeros(3)

for n in range(3):
    f_in = '../data_prep/mask1/masked_sample_' + str(n+1) + '.dat'
    # f_in = '../data_prep/mask2/masked_sample_' + str(n+1) + '.dat'
   
    df = pd.read_csv(f_in,sep="\t",header = None)
    df.columns = ['r','th','ph']
    R_max[n] = df['r'].max()
    R_min[n] = df['r'].min()

    for l in range(50):
        f_samp = 'Randomized_data/s'+str(n+1)+'_rand'+str(l+1)+'.dat'
        f_out = 'anis_s' + str(n+1) + '_rand' + str (l+1) + '.dat'
        
        compute_renyi_entropy(f_samp,f_out,R_min[n],R_max[n])

In [3]:
# calculation of normalized entropy dispersion
for n in range(3):
    criteria = []

    for f in range(nf):
        file = 'anis_s'+ str(n+1) + '_rand' + str (f+1) + '.dat' 
        data = np.loadtxt(file)
        RR = data[:, 0]
        samples = data[:, 1:6]  # columns a1 to a5

        a_mean = np.mean(samples, axis=1)
        stab_cri = np.std(samples, axis=1, ddof=0)
        frac_cri = stab_cri / a_mean
        criteria.append(frac_cri)

    criteria = np.array(criteria)
    mean_crit = np.mean(criteria, axis=0)
    d_crit = np.std(criteria, axis=0, ddof=1)
    
    df_out = pd.DataFrame({'r': RR, 'crit': mean_crit, 'sd': d_crit})
    df_out.to_csv('sample_rand_'+str(n+1)+'_criteria.csv' , index=False)


In [4]:
# remove unwanted files
for n in range(3):
    for l in range(nf):
        file_name = 'anis_s' + str(n+1) + '_rand' + str (l+1) + '.dat'
        os.remove(file_name)
        
import shutil
shutil.rmtree('Randomized_data')
   